2-1. Nate 뉴스기사 제목 스크래핑하기

In [ ]:
# https://news.nate.com/recent?mid=n0100
# 최신뉴스, 정치 , 경제, 사회, 세계, IT/과학 
# 6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기

# Image, 기사제목, 기사링크

# 뉴스기사의 Image를 출력 하세요 
# 1) Image의 도메인이름이 포함된 경로와 src 속성의 경로를 합치려면 urljoin 함수를 사용하세요.
#     from urllib.parse import urljoin
    
#     url = ‘https://news.nate.com/recent?mid=n0100’
#     src=’ //thumbnews.nateimg.co.kr/news90///news.nateimg.co.kr/orgImg/na/2025/07/23/7408335_high.jpg’

# 2) Image 출력은 Image 클래스와 display 함수를 사용하세요.
#     from IPython.display import Image, display

# 3) img 엘리먼트의 존재 여부를 체크하신 후에 src 속성의 이미지를 경로를 추출하기
#   => Image 가 없는 뉴스도 있기 때문에 
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

#요청헤더
req_header = {

    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36" 
}
section_dict = {'최신뉴스':'recent?mid=n0100','정치':'section?mid=n0200','경제':'section?mid=n0300',
                '사회':'section?mid=n0400','세계':'section?mid=n0500','IT/과학':'section?mid=n0600'}



def print_news_image(section_name):
    sid = section_dict[section_name]
    url = f'https://news.nate.com/{sid}'
    #응답 엔코딩 헤더 설정
    res = requests.get(url,headers=req_header)
    res.encoding = 'euc-kr'
    
    if res.ok:
        html = res.text
        soup = BeautifulSoup(html,'html.parser')
        

        a_tags = soup.select("div.mlt01 a[href*='//news.nate.com/view/']")
        #3) img 엘리먼트의 존재 여부를 체크
        for a_tag in a_tags: #img_tag의 타입은 Tag <img>
            #Title추출
            h2_tags = a_tag.select_one("span.tb h2.tit")
            
            print(f'기사제목 : {h2_tags.text}')

            img_tags = a_tag.select("img[src*='//thumbnews.nateimg.co.kr/news90']")
            for img_tag in img_tags:
                if img_tag:
                    img_url = img_tag['src']
                    #1) Image의 도메인이름이 포함된 경로와 src 속성의 경로를 합치려면 urljoin 함수를 사용하세요.
                    join_url = urljoin(url,img_url) #기준(base) URL과 상대(relative) URL을 합쳐 절대 URL을 생성

                    print(f'기사링크 : {join_url}')
                    #2) Image 출력은 Image 클래스와 display 함수를 사용
                    print(f'Image : ')
                    display(Image(join_url))
                else :
                    print("이미지를 찾을 수 없습니다.")



            
    else:
        print(f'Error Code = {res.status_code}')

for section in section_dict:
    print(f'========={section} 입니다.=========')
    print_news_image(section)

2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 

In [ ]:
# :  하나의 웹툰의 제목(title)과 회차번호(no),회차의URL(url) 을 입력으로 받는 함수를 선언합니다. 
#    def download_one_episode(title,no,url):

# 아래와 같이 호출합니다.
# download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

# img\일렉시드\341 디렉토리가 생성되며 , 
# 그 디렉토리 아래에 웹툰 image들이 다운로드 되도록 해주세요.

import requests
from bs4 import BeautifulSoup
import os

def download_one_episode(title,no,url):
    imgdir_name = os.path.join('img', title, str(no))
    req_header = {'referer': url}

    # 이미지 저장 폴더가 없으면 생성
    os.makedirs(imgdir_name, exist_ok = True)
    res = requests.get(url)
    if not res.ok:
        print(f'Error Code = {res.status_code}')
        exit()
    # 이미지 URL 추출
    soup = BeautifulSoup(res.text, 'html.parser')

    img_url_list = [img_tag['src'] for img_tag in soup.select("img[src *= IMAG01]")]
    #print(img_url_list)
    #이미지 다운로드
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_path = os.path.join(imgdir_name, os.path.basename(img_url))
            with open(file_path, 'wb') as file:
                #print(f'Wrting to {file_path} ({len(img_data):,} bytes)')
                file.write(img_data)

        else:
            print(f'Error Code = {res.status_code}')
    print(f"네이버 웹툰의 '{title}' 만화 {no}회를 다운로드 하였습니다.")
    print(f'{imgdir_name} 폴더에 있습니다.')
#호출
download_one_episode('일렉시드',341,'https://comic.naver.com/webtoon/detail?titleId=717481&no=341&week=wed')

네이버 웹툰의 '일렉시드' 만화 341회를 다운로드 하였습니다.
img\일렉시드\341에 있습니다.
